# ResNext Model for Chest XRAY Catheter and Line Detection Challenge

**Description of challenge**: Classify the presence and correct placement of tubes on chest x-rays to save lives

**Dataset**: CLiP, catheter and line position dataset[1]

**Model Architecture**: resnext50_32x4d

**Performance**: 93% accuracy using 4-fold cross-validation (comparable to state of the art models in literature[2])

**Explainability**: Using occlusion-based attribution to see regions of high importance to the model's decision-making process, thus adding interpretability

**Purpose**: This notebook goes through the steps for explaining this model. It is accompanied by the resnext-training.ipynb notebook which shows the training and evaluating process.  

**Notes for use**: notebook assumes kaggle environment and .cuda() availability. 

*[1]: https://www.nature.com/articles/s41597-021-01066-8.pdf*

*[2]: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8017400/pdf/ryai.2020190082.pdf*

# 1 - Setup

In [2]:
!pip install gdown
!pip install captum
!pip install timm
!gdown 1dWwapuVx_aK6K2j5Jj1St2F7RTQ9DirG
!gdown 1g-pijm1Db9cfOxBhHw5h5aqVnCEWjtTL #this is cvc-normal from train set
!gdown 1CalZo9FPVKDHZ-66pe2CqWoZm94X4j8m #this is cvc-abnormal from train set predicted incorrectly
!gdown 1mjBY-yXXIzZhaLCAC5dfxBshF68GjCMM #cvc-abnormal train set

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from PIL import Image

import os
import json
import numpy as np
from matplotlib.colors import LinearSegmentedColormap
import pandas as dp

import torchvision
from torchvision import models
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset

from captum.attr import IntegratedGradients
from captum.attr import GradientShap
from captum.attr import Occlusion
from captum.attr import NoiseTunnel
from captum.attr import visualization as viz
import timm

from albumentations import (
    Compose, OneOf, Normalize, Resize, RandomResizedCrop, RandomCrop, HorizontalFlip, VerticalFlip, 
    RandomBrightness, RandomContrast, RandomBrightnessContrast, Rotate, ShiftScaleRotate, Cutout, 
    IAAAdditiveGaussianNoise, Transpose
    )
from albumentations.pytorch import ToTensorV2
from albumentations import ImageOnlyTransform

%cd ..


In [6]:
class CustomResNext(nn.Module):
    def __init__(self, model_name='resnext50_32x4d', pretrained=False):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained)
        n_features = self.model.fc.in_features
        self.model.fc = nn.Linear(n_features, 11)

    def forward(self, x):
        x = self.model(x)
        return x

model = CustomResNext()
if torch.cuda.is_available():
    model.cuda()
weights = torch.load('working/resnext50_32x4d_fold2_best-21aug-7hrs.pth')
model.load_state_dict(weights, strict=False)
model.eval()

target_cols=['ETT - Abnormal', 'ETT - Borderline', 'ETT - Normal',
             'NGT - Abnormal', 'NGT - Borderline', 'NGT - Incompletely Imaged', 'NGT - Normal', 
             'CVC - Abnormal', 'CVC - Borderline', 'CVC - Normal',
             'Swan Ganz Catheter Present']

In [7]:
#CVC - abnormal 
img = Image.open('working/1.2.826.0.1.3680043.8.498.68286643202323212801283518367144358744.jpg').convert('RGB')
normalize_transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize(600),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean = (0.485, 0.456, 0.406), 
                                     std = (0.229, 0.224, 0.225))])


input = normalize_transform(img).unsqueeze(0)
print(input.shape)

In [8]:
output = model(input.cuda())
output = F.softmax(output, dim=1)
prediction_score, pred_label_idx = torch.topk(output, 1)
predicted_label = target_cols[pred_label_idx.item()]
print('Predicted:', predicted_label, '(', prediction_score.squeeze().item(), ')')


In [36]:
# To clear GPU memory - use only if you need to restart
from numba import cuda
cuda.select_device(0)
cuda.close()
cuda.select_device(0)

## 2- Gradient-based attribution

In [10]:
occlusion = Occlusion(model)

attributions_occ = occlusion.attribute(input.cuda(),
                                       strides = (3, 30, 30),
                                       target=pred_label_idx,
                                       sliding_window_shapes=(3,45, 45),
                                       baselines=0,
                                       show_progress=True)

_ = viz.visualize_image_attr_multiple(np.transpose(attributions_occ.squeeze().cpu().detach().numpy(), (1,2,0)),
                                      np.transpose(input.squeeze().cpu().detach().numpy(), (1,2,0)),
                                      ["original_image", "heat_map"],
                                      ["all", "positive"],
                                      show_colorbar=True,
                                      outlier_perc=2,
                                     )


In [11]:
occlusion = Occlusion(model)

attributions_occ = occlusion.attribute(input.cuda(),
                                       strides = (3, 50, 50),
                                       target=pred_label_idx,
                                       sliding_window_shapes=(3,60, 60),
                                       baselines=0,
                                       show_progress=True)

_ = viz.visualize_image_attr_multiple(np.transpose(attributions_occ.squeeze().cpu().detach().numpy(), (1,2,0)),
                                      np.transpose(input.squeeze().cpu().detach().numpy(), (1,2,0)),
                                      ["original_image", "heat_map"],
                                      ["all", "positive"],
                                      show_colorbar=True,
                                      outlier_perc=2,
                                     )